In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [6]:
Idle_data1 = pd.read_csv('..\HandTrackingData\Idle1.csv')
Idle_data1 = Idle_data1.to_numpy()

Idle_data2 = pd.read_csv('..\HandTrackingData\Idle2.csv')
Idle_data2 = Idle_data2.to_numpy()

Idle_data3 = pd.read_csv('..\HandTrackingData\Idle3.csv')
Idle_data3 = Idle_data3.to_numpy()

Bow_data = pd.read_csv('..\HandTrackingData\Bow.csv')
Bow_data = Bow_data.to_numpy()

Sword_data = pd.read_csv('..\HandTrackingData\Sword.csv')
Sword_data = Sword_data.to_numpy()

Pistol_data = pd.read_csv('..\HandTrackingData\Pistol.csv')
Pistol_data = Pistol_data.to_numpy()

MachineGun_data = pd.read_csv('..\HandTrackingData\MachineGun.csv')
MachineGun_data = MachineGun_data.to_numpy()

Spear_data = pd.read_csv('..\HandTrackingData\Spear.csv')
Spear_data = Spear_data.to_numpy()

Granade_data = pd.read_csv('..\HandTrackingData\Grenade.csv')
Granade_data = Granade_data.to_numpy()

print(Idle_data1.shape)
print(Idle_data2.shape)
print(Idle_data3.shape)
print(Bow_data.shape)
print(Sword_data.shape)
print(Pistol_data.shape)
print(MachineGun_data.shape)
print(Spear_data.shape)
print(Granade_data.shape)

(2000, 115)
(2000, 115)
(2000, 115)
(4000, 115)
(4000, 115)
(4000, 115)
(4000, 115)
(4000, 115)
(4000, 115)


In [7]:
Idle_data_Seq = []
Bow_data_Seq= []
Sword_data_Seq = []
Pistol_data_Seq = []
MachineGun_data_Seq = []
Spear_data_Seq = []
Granade_data_Seq = []


Objects = ['Idle', 'Bow','Sword', 'Pistol', 'MachineGun', 'Spear', 'Granade']
WindowSize = 80

In [8]:

def Create_Sequences(data, window_size):
    """
    주어진 데이터와 윈도우 크기를 기반으로 시퀀스를 생성합니다.

    Args:
        data (list or np.ndarray): 입력 데이터
        window_size (int): 시퀀스의 윈도우 크기

    Returns:
        np.ndarray: 생성된 시퀀스 배열
    """
    
    Sequences = []
    for seq in range(len(data) - window_size)+1:
        Sequences.append(data[seq:seq + window_size])
    return np.array(Sequences)

In [6]:
Idle_data_Seq = Create_Sequences(Idle_data1, WindowSize)
Bow_data_Seq = Create_Sequences(Bow_data, WindowSize)
Sword_data_Seq = Create_Sequences(Sword_data, WindowSize)
Pistol_data_Seq = Create_Sequences(Pistol_data, WindowSize)
MachineGun_data_Seq = Create_Sequences(MachineGun_data, WindowSize)
Spear_data_Seq = Create_Sequences(Spear_data, WindowSize)
Granade_data_Seq = Create_Sequences(Granade_data, WindowSize)

In [ ]:
print(Bow_data_Seq.shape)
print(Sword_data_Seq.shape)
print(Pistol_data_Seq.shape)
print(MachineGun_data_Seq.shape)
print(Spear_data_Seq.shape)
print(Granade_data_Seq.shape)

In [ ]:
ConcatenatedSeqData = np.concatenate([Bow_data_Seq, Sword_data_Seq , Pistol_data_Seq, MachineGun_data_Seq, Spear_data_Seq, Granade_data_Seq], axis=0)

ConcatenatedSeqData.shape

In [ ]:
X_data = ConcatenatedSeqData[:, :, :-1]
Labels = ConcatenatedSeqData[:, 0, -1]

print(X_data.shape)
print(X_data)

print(Labels.shape)
print(Labels)

In [ ]:
from tensorflow.keras.utils import to_categorical

Y_data = to_categorical(Labels, num_classes=len(Objects))

print(Y_data)

In [ ]:
from sklearn.model_selection import train_test_split

X_data = X_data.astype(np.float32)
Y_data = Y_data.astype(np.float32)

X_train, X_val, Y_train, Y_val = train_test_split(X_data, Y_data, test_size=0.2, random_state=2024)

print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='tanh', input_shape=X_train.shape[1:3]),
    Dense(32, activation='tanh'),
    Dense(len(Objects), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    X_train,
    Y_train,
    validation_data=(X_val, Y_val),
    epochs=100,
    callbacks=[
        ModelCheckpoint('models/model6.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
import tensorflow as tf

# .h5 모델 로드
h5_model_path = "models\model6.h5"
saved_model_path = "models\model6_tf"

model = tf.keras.models.load_model(h5_model_path)

# SavedModel 형식으로 저장
tf.saved_model.save(model, saved_model_path)

print(f"SavedModel saved at: {saved_model_path}")

#tf python -m tf2onnx.convert --saved-model C:\Project\saved_model --output C:\Project\model2.onnx
#python -m tf2onnx.convert --saved-model tensorflow-model-path --output model.onnx
#python -m tf2onnx.convert --input model3.h5 --output model33.onnx
#python -m tf2onnx.convert --saved-model path/to/saved_model --output model.onnx --input-shape "1,80,138"

In [ ]:
import tf2onnx
import tensorflow as tf

# TensorFlow SavedModel 경로 및 변환된 ONNX 모델 저장 경로
saved_model_path = "models\model6.h5"
onnx_model_path = "models\model6.onnx"

# 변환에 사용할 입력 shape
input_signature = [tf.TensorSpec([1, 80, 114], tf.float32, name="input")]

# SavedModel 로드
model = tf.keras.models.load_model(saved_model_path)

# ONNX로 변환
offset = 0
onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=input_signature,
    opset=13  # ONNX opset 버전 지정 (13은 일반적으로 사용되는 버전)
)

# ONNX 모델 저장
with open(onnx_model_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"ONNX model saved at: {onnx_model_path}")

In [ ]:
import onnx

# onnx 파일 구조 파악.
onnx_model = onnx.load("models\model6.onnx")
onnx.checker.check_model(onnx_model)

print(onnx.helper.printable_graph(onnx_model.graph))